## Classifier

In [10]:
import numpy as np
import tensorflow as tf
from utils.general_utils import test_all_close

### Compute the 

In [11]:
def softmax(x):
    
    """Args:
        x: tf.Tensor with shape (n_samples, n_features). Note feature vectors are
            feature vectors are
            represented by row-vectors. (For simplicity, no need to handle 1-d)
    Returns:
        out: tf.Tensor with shape (n_sample, n_features).
    """
    x_max = tf.reduce_max(x, 1, keep_dims=True)
    
    x_sub = tf.subtract(x, x_max)
    
    x_exp = tf.exp(x_sub)
    
    sum_exp = tf.reduce_sum(x_exp, 1, keep_dims=True)
    out = tf.div(x_exp, sum_exp)
    
    return out

### Compute the cross entropy loss in tf

In [27]:
def cross_entropy_loss(y, yhat):
    """The loss should be summed over the current minibatch
    
    y is a one hot tensor of shape (n_samples, n_classes) and yhat is a tensor
    
    of shape (n_samples, n_classes). y should be of dtype tf.int32, and yhat should
    tf.float32.
    
    Args:
        y: tf.Tensor with shape(n_samples, n_classes). One-hot encoded.
        yhat: tf.Tensor with shape(n_samples, n_classes). Each row encodes a
                probability distribution and should sum to 1.
        
    Return:
        out: tf.Tensor with shape (1, ) (Scalar output)
    """
    l_yhat = tf.log(yhat)
    product = tf.multiply(tf.to_float(y), l_yhat)
    
    out = tf.negative(tf.reduce_sum(product))
    
    return out

### Some simple tests of softmax

In [28]:
test1 = softmax(tf.constant(np.array([[1001, 1002], [3, 4]]), dtype=tf.float32))
with tf.Session() as sess:
    test1 = sess.run(test1)
print(test1)

[[ 0.26894143  0.7310586 ]
 [ 0.26894143  0.7310586 ]]


In [29]:
x = np.array([[1001, 1002], [3, 4]])

In [30]:
x

array([[1001, 1002],
       [   3,    4]])

### Tests of cross entropy loss

In [31]:
y = np.array([[0, 1], [1, 0], [1, 0]])
yhat = np.array([[.5, .5], [.5, .5], [.5, .5]])

test1 = cross_entropy_loss(
        tf.constant(y, dtype=tf.int32),
        tf.constant(yhat, dtype=tf.float32)
)

In [32]:
with tf.Session() as sess:
    test1 = sess.run(test1)
expected = -3 * np.log(-5)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
print(test1, expected)

2.07944 nan


## Q2 initialization

In [34]:
import numpy as np
import tensorflow as tf

In [35]:
def xavier_weight_init():
    """
    Returns function that creates random tensor.
    
    The specified function will take in a shape (tuple or 1-d array)
    anh returns a random tensor of the specified shape drawn form the Xavier initilization distribution
    """
    
    def _xavier_initializer(shape, **kwargs):
        """
        Defines an initializer for the Xavier distribution.
        Specifically, the output should be sampled uniformly from
        [-epsilon, epsilon] where
            epsilon = sqrt(6) / <sum of the sizes of shape's dimensions>
            
        This function will be used as a variable initializer.
        
        Returns:
            out: tf.Tensor of specified shape sampled from the Xavier distribution.
        """
        
        epsilon = np.sqrt(6 / np.sum(shape))
        out = tf.Variable(tf.random_uniform(shape=shape, minval=-epsilon, maxval=epsilon))
        
        return out
    
    return _xavier_initializer

### Test initialization basic

In [36]:
xavier_initializer = xavier_weight_init()
shape = (1,)
xavier_mat = xavier_initializer(shape)

In [37]:
print(xavier_mat)

<tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>


In [38]:
shape = (1, 2, 3)
xavier_mat = xavier_initializer(shape)
print(xavier_mat)

<tf.Variable 'Variable_1:0' shape=(1, 2, 3) dtype=float32_ref>


## Q2 partial parse

In [56]:
class PartialParse(object):
    def __init__(self, sentence):
        """
        Initializers this partial parse
        
        Your code should initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the last element of the list
            
            self.buffer: The current buffer represented as a list with the first item on the the buffer as the first item of the list
            
            self.dependencies: The list of dependencies producted so far.
            Represented as a list of tuples where each tuple is of the form 
            
            
        The root token should be represented with the string "ROOT" 
        
        Args:
            sentence: The sentence to be parsed as a list of words.
                Your code should not modify the sentence.
                
        """
        self.sentence = sentence
        
        self.stack = ['ROOT']
        self.buffer = sentence[:]
        self.dependencies = []
    
    def parse_step(self, transition):
        """
        Performs a single parse step by applying the given 
        transition to this partial parse
        
        Args:
            transition: A string that equals "S", "LA", or "RA"
            representing the shift, left-arc, and right-arc transitions. 
        """
        if transition == "S":
            self.stack.append(self.buffer[0])
            self.buffer.pop(0)
        elif transition == "LA":
            self.dependencies.append((self.stack[-1], self.stack[-2]))
            self.stack.pop(-2)
        else:
            self.dependencies.append((self.stack[-2], self.stack[-1]))
            self.stack.pop(-1)
    
    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse
        
        Args:
            transitions: The list of transitions in the order they should be applied 
        
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
             as a list of tuples where each tuple is of the form
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies
            

### Parses a list of sentences in minibatches using a model

In [57]:
def minibatch_parse(sentences, model, batch_size):
    """
    Args:
        sentences: A list of sentences to be parsed (each sentence is a list of words)
        
        model: The model that makes parsing decisions. It is assumed to have a function
            model.predict(partial_parses) that takes in a list of PartialParses as input and
            and returns a list of transitions predicted for each parse. 
            That is, after calling transitions = model. predict(partial_parses)
            transitions[i] will be the next transition to apply to partial_parse[i]
            
        batch_size: The number of PartialParses to include in each minibatch
        
    Returns:
        dependencies: A list where each element is the dependencies list for a parsed sentence
        Ordering should be the same as in sentences
    """
    partial_parses = [PartialParse(s) for s in sentences]
    
    unfinished_parse = partial_parses
    
    while len(unfinished_parse) > 0:
        minibatch = unfinished_parse[0:batch_size]
        
        while len(minibatch) > 0:
            transitions = model.predict(minibatch)
            for index, action in enumerate(transitions):
                minibatch[index].parse_step(action)
            minibatch = [parse for parse in minibatch if len(parse.stack) > 1 or len(parse.buffer) > 0]
            
        # move to the next batch
        unfinished_parse = unfinished_parse[batch_size:]
    
    dependencies = []
    for n in range(len(sentences)):
        dependencies.append(partial_parses[n].dependencies)
        
    return dependencies

### Tests that a single parse step returns the expected output

In [58]:
def test_step(name, transition, stack, buf, deps, ex_stack, ex_buf, ex_deps):
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps
    
    pp.parse_step(transition)
    
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    print(stack)
    print(buf)
    print(deps)
    print(stack==ex_stack, buf==ex_buf, deps==ex_deps)

In [59]:
test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [], ("ROOT", "the", "cat"), ("sat",), ())

('ROOT', 'the', 'cat')
('sat',)
()
True True True


In [60]:
test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], []
         , ("ROOT", "cat",), ("sat",), (("cat", "the"),))

('ROOT', 'cat')
('sat',)
(('cat', 'the'),)
True True True


In [61]:
test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [], 
         ("ROOT", "run",), (), (("run", "fast"),))

('ROOT', 'run')
()
(('run', 'fast'),)
True True True


### Simple tests for the PartialParse.parse function

In [62]:
sentence = ["parse", "this", "sentence"]
dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
dependencies = tuple(sorted(dependencies))
expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))

In [63]:
print(dependencies)
print(expected)
print(tuple(sentence))

(('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
(('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
('parse', 'this', 'sentence')


### Dummy model for testing the minibatch parse function

In [64]:
class DummyModel:
    """
    First shifts everything onto the stack ad then does exclusively right arcs if the first
    word of the sentence is "right", "left" if otherwise.
    """
    
    def predict(self, partial_parses):
        return [("RA" if pp.stack[1] is "right" else "LA") if len(pp.buffer) == 0 else "S"
               for pp in partial_parses]
    

In [65]:
sentences = [["right", "arcs", "only"],
            ["right", "arcs", "only", "again"],
            ["left", "arcs", "only"],
            ["left", "arcs", "only", "again"]]
deps = minibatch_parse(sentences, DummyModel(), 2)

In [66]:
deps = tuple(sorted(deps))

In [67]:
deps[0]

[('again', 'only'), ('again', 'arcs'), ('again', 'left'), ('again', 'ROOT')]

In [68]:
deps[1]

[('arcs', 'only'), ('right', 'arcs'), ('ROOT', 'right')]

In [69]:
class Model(object):
    """Abstracts a Tensorflow graph for a learning task.

    We use various Model classes as usual abstractions to encapsulate tensorflow
    computational graphs. Each algorithm you will construct in this homework will
    inherit from a Model object.
    """

    def add_placeholders(self):
        """Adds placeholder variables to tensorflow computational graph.

        Tensorflow uses placeholder variables to represent locations in a
        computational graph where data is inserted.  These placeholders are used as
        inputs by the rest of the model building and will be fed data during
        training.

        See for more information:
        https://www.tensorflow.org/versions/r0.7/api_docs/python/io_ops.html#placeholders
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def create_feed_dict(self, inputs_batch, labels_batch=None):
        """Creates the feed_dict for one step of training.

        A feed_dict takes the form of:
        feed_dict = {
                <placeholder>: <tensor of values to be passed for placeholder>,
                ....
        }

        If labels_batch is None, then no labels are added to feed_dict.

        Hint: The keys for the feed_dict should be a subset of the placeholder
                    tensors created in add_placeholders.

        Args:
            inputs_batch: A batch of input data.
            labels_batch: A batch of label data.
        Returns:
            feed_dict: The feed dictionary mapping from placeholders to values.
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def add_prediction_op(self):
        """Implements the core of the model that transforms a batch of input data into predictions.

        Returns:
            pred: A tensor of shape (batch_size, n_classes)
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def add_loss_op(self, pred):
        """Adds Ops for the loss function to the computational graph.

        Args:
            pred: A tensor of shape (batch_size, n_classes)
        Returns:
            loss: A 0-d tensor (scalar) output
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def add_training_op(self, loss):
        """Sets up the training Ops.

        Creates an optimizer and applies the gradients to all trainable variables.
        The Op returned by this function is what must be passed to the
        sess.run() to train the model. See

        https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#Optimizer

        for more information.

        Args:
            loss: Loss tensor (a scalar).
        Returns:
            train_op: The Op for training.
        """

        raise NotImplementedError("Each Model must re-implement this method.")

    def train_on_batch(self, sess, inputs_batch, labels_batch):
        """Perform one step of gradient descent on the provided batch of data.

        Args:
            sess: tf.Session()
            input_batch: np.ndarray of shape (n_samples, n_features)
            labels_batch: np.ndarray of shape (n_samples, n_classes)
        Returns:
            loss: loss over the batch (a scalar)
        """
        feed = self.create_feed_dict(inputs_batch, labels_batch=labels_batch)
        _, loss = sess.run([self.train_op, self.loss], feed_dict=feed)
        return loss

    def predict_on_batch(self, sess, inputs_batch):
        """Make predictions for the provided batch of data

        Args:
            sess: tf.Session()
            input_batch: np.ndarray of shape (n_samples, n_features)
        Returns:
            predictions: np.ndarray of shape (n_samples, n_classes)
        """
        feed = self.create_feed_dict(inputs_batch)
        predictions = sess.run(self.pred, feed_dict=feed)
        return predictions

    def build(self):
        self.add_placeholders()
        self.pred = self.add_prediction_op()
        self.loss = self.add_loss_op(self.pred)
        self.train_op = self.add_training_op(self.loss)


In [72]:
import os
import time
import tensorflow as tf
import pickle